# Data Vault Demo (Dev -- Full Access)

The basic concept of the Data Vault is that when a user authenticates themself, they receive an engine that gives them access to all the data (rows, columns, tables, schema, etc.) for which they are authorized.  Users who can authenticate themselves for multiple roles can use those roles simultaneously.  We are keeping in mind the importance of Data Lineage Management (tracked by issue https://github.com/os-climate/os_c_data_commons/issues/50) but is not treated as part of this particular prototype.

The steps of this demo are:

1. **Authenticate and acquire SQLAlchemy engine**
    1. **Dev engine sees all**
    2. Quant engine can do temp scoring but not see fundamental company info
    3. User engine can use temp scoring but not see cumulative emissions nor overshoot info
2. **With Dev engine, construct Vaults for:**
    1. **Fundamental corporate financial information**
    2. **Corporate emissions data (base year, historical)**
    3. **Corporate target data (start year, end year, target start value, target end value)**
    4. **Sector benchmark data (production, CO2e intensity)**
3. **Dev Engine: Visualize projected emissions (targets and trajectories) and calculate cumulative emissions**
4. Quant Engine: Using calculated cumulative emmisions, visualize per-company trajectory and target temperature scores
5. User Engine: Using consensus probability scoring and own portfolio data (ISIN, position value)
    1. Calculate publishable per-company temperature alignment score
    2. Based on aggregate corporate and portfolio information, produce weighting scores to yield overall portfolio alignment score

In [1]:
import os
import pathlib
from dotenv import load_dotenv
import osc_ingest_trino as osc
import trino
from sqlalchemy.engine import create_engine

import json
import pandas as pd
import numpy as np
from numpy.testing import assert_array_equal
import ITR

# from ITR.portfolio_aggregation import PortfolioAggregationMethod
# from ITR.temperature_score import TemperatureScore
# from ITR.configs import ColumnsConfig, TemperatureScoreConfig
# from ITR.data.data_warehouse import DataWarehouse
from ITR.data.vault_providers import VaultCompanyDataProvider, VaultProviderProductionBenchmark, \
    VaultProviderIntensityBenchmark, DataVaultWarehouse, requantify_df

# from ITR.interfaces import ICompanyData, EScope, ETimeFrames, PortfolioCompany, IEIBenchmarkScopes, \
#     IProductionBenchmarkScopes
from ITR.interfaces import EScope, IProductionBenchmarkScopes, IEIBenchmarkScopes

from ITR.data.osc_units import ureg, Q_, PA_

from pint_pandas import PintArray

using connect string: trino://MichaelTiemannOSC@trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443/osc_datacommons_dev/demo_dv


In [2]:
# Load some standard environment variables from a dot-env file, if it exists.
# If no such file can be found, does not fail, and so allows these environment vars to
# be populated in some other way
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

### Step 1: Initialize Vault user 'Dev', which has full visibility into corporate financial, production, and target data

In [3]:
sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ['TRINO_USER'],
    host = os.environ['TRINO_HOST'],
    port = os.environ['TRINO_PORT']
)

ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'demo_dv'
itr_prefix = 'template_'

sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    'http_scheme': 'https',
    'catalog': ingest_catalog,
    'schema': ingest_schema,
}

engine_dev = create_engine(sqlstring, connect_args = sqlargs)
print("connecting with engine " + str(engine_dev))
qres = osc._do_sql(f"show tables in {ingest_schema}", engine_dev, verbose=True)

# Check that we have the tables we need
required_tables = ['company_data', 'target_data', 'trajectory_data', 'emissions_data']
existing_tables = [ q[0] for q in qres ]
missing_tables = [ rtable for rtable in required_tables if f"{itr_prefix}{rtable}" not in existing_tables ]
if missing_tables:
    print(f"Missing tables (itr_prefix = {itr_prefix}): {missing_tables}")
    assert False

connecting with engine Engine(trino://MichaelTiemannOSC@trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443/)
show tables in demo_dv
[('benchmark_ei',), ('benchmark_prod',), ('rmi_benchmark_ei',), ('rmi_benchmark_prod',), ('rmi_company_data',), ('rmi_cumulative_budget_1',), ('rmi_cumulative_emissions',), ('rmi_emissions_data',), ('rmi_overshoot_ratios',), ('rmi_production_data',), ('rmi_target_data',), ('rmi_temperature_scores',), ('rmi_trajectory_data',), ('template_benchmark_ei',), ('template_benchmark_prod',), ('template_company_data',), ('template_cumulative_budget_1',), ('template_cumulative_budgets',), ('template_cumulative_emissions',), ('template_emissions_data',), ('template_overshoot_ratios',), ('template_production_data',), ('template_target_data',), ('template_temperature_scores',), ('template_trajectory_data',)]


### The ITR module provides Vault objects that coordinate the interaction of Dev, Quant, and User roles.

The SQLAlchemy engines mediate the actual interaction with the Data Vault.

### Step 2: construct vaults for corporate financial, production, and target information.

We also create benchmark data (which is presumed public information).  There's more work to be done to modularly add new benchmarks that automatically become available options to to the ITR tool.

In this demo we read ITR benchmark data from JSON files (REST API-friendly).  Such data coming from the notebook filesystem is "untethered" data.  The corporate data comes from an existing data pipeline (in this case, the pipeline processing RMI data).  When data comes from the data commons, it is "tethered" to the Data Commons.  The Data Vault can only control access to data that goes through the Data Commons via 'engines'.

In [4]:
root = root = os.path.dirname(os.getcwd())
benchmark_prod_json = os.path.join(root, "examples/data", "json-units", "benchmark_production_OECM.json")
benchmark_EI_json = os.path.join(root, "examples/data", "json-units", "benchmark_EI_OECM_S3.json")

# load production benchmarks
with open(benchmark_prod_json) as json_file:
    parsed_json = json.load(json_file)
prod_bms = IProductionBenchmarkScopes.parse_obj(parsed_json)
vault_production_bm = VaultProviderProductionBenchmark(engine=engine_dev, benchmark_name=f"{itr_prefix}benchmark_prod", production_benchmarks=prod_bms)

# load intensity benchmarks
with open(benchmark_EI_json) as json_file:
    parsed_json = json.load(json_file)
ei_bms = IEIBenchmarkScopes.parse_obj(parsed_json)
vault_EI_bm = VaultProviderIntensityBenchmark(engine=engine_dev, benchmark_name=f"{itr_prefix}benchmark_ei", EI_benchmarks=ei_bms)

# load company data
# TODO: Pandas reads null data mixed with integers as float64 (np.nan).  This can be fixed post hoc with astype('Int16')
vault_company_data = VaultCompanyDataProvider(engine=engine_dev, company_table=f"{itr_prefix}company_data")

vault_warehouse = DataVaultWarehouse(engine_dev, vault_company_data, vault_production_bm, vault_EI_bm, itr_prefix=itr_prefix)

/Users/michael/opt/miniconda3/envs/pandas_2/lib/python3.11/site-packages/pandas/core/dtypes/missing.py:230: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return _isna_array(np.asarray(obj), inf_as_na=inf_as_na)
EXECUTE IMMEDIATE not available for trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443; defaulting to legacy prepared statements (TrinoUserError(type=USER_ERROR, name=SYNTAX_ERROR, message="line 1:19: mismatched input ''SELECT 1''. Expecting: 'USING', <EOF>", query_id=20230730_202703_04222_65cw7))


constructed fully qualified table name as: "demo_dv.template_benchmark_prod"
inserting 2208 records
  (2019, 0.0, 'dimensionless', 'Global', 'Steel', 'AnyScope')
  (2020, 0.00306, 'dimensionless', 'Global', 'Steel', 'AnyScope')
  (2021, 0.00306, 'dimensionless', 'Global', 'Steel', 'AnyScope')
  ...
  (2050, -0.037, 'dimensionless', 'North America', 'Oil & Gas', 'AnyScope')
batch insert result: [(2208,)]


/Users/michael/opt/miniconda3/envs/pandas_2/lib/python3.11/site-packages/pandas/core/dtypes/missing.py:230: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return _isna_array(np.asarray(obj), inf_as_na=inf_as_na)


constructed fully qualified table name as: "demo_dv.template_benchmark_ei"
inserting 5000 records
  (2019, 0.574, 'CO2e / Fe', 'Global', 'Steel', 'S1', 396, 'CO2 * gigametric_ton', 1.5, 'delta_degree_Celsius')
  (2020, 0.54, 'CO2e / Fe', 'Global', 'Steel', 'S1', 396, 'CO2 * gigametric_ton', 1.5, 'delta_degree_Celsius')
  (2021, 0.509, 'CO2e / Fe', 'Global', 'Steel', 'S1', 396, 'CO2 * gigametric_ton', 1.5, 'delta_degree_Celsius')
  ...
  (2026, 2.106, 'CO2e * megametric_ton / CH4 / bcm', 'Global', 'Gas', 'S3', 396, 'CO2 * gigametric_ton', 1.5, 'delta_degree_Celsius')
batch insert result: [(5000,)]
inserting 5000 records
  (2027, 2.099, 'CO2e * megametric_ton / CH4 / bcm', 'Global', 'Gas', 'S3', 396, 'CO2 * gigametric_ton', 1.5, 'delta_degree_Celsius')
  (2028, 2.093, 'CO2e * megametric_ton / CH4 / bcm', 'Global', 'Gas', 'S3', 396, 'CO2 * gigametric_ton', 1.5, 'delta_degree_Celsius')
  (2029, 2.086, 'CO2e * megametric_ton / CH4 / bcm', 'Global', 'Gas', 'S3', 396, 'CO2 * gigametric_ton', 

2023-07-30 16:28:00,477 - ITR.data.vault_providers - ERROR - Provide either historic emissions data or projections for companies with IDs [('OXY CHEM', 'US6745991058-chem'), ('Synthomer Plc', 'GB0009887422'), ('Covestro AG', 'DE0006062144'), ('OXY OIL AND GAS', 'US6745991058-oil')]


[('OXY CHEM', 'US6745991058-chem'), ('Synthomer Plc', 'GB0009887422'), ('Covestro AG', 'DE0006062144'), ('OXY OIL AND GAS', 'US6745991058-oil')]

create table demo_dv.template_cumulative_emissions with (
    format = 'ORC',
    partitioning = array['scope']
) as
select C.company_name, C.company_id, 'demo_dv' as source, 'S1S2' as scope, P.year,
       sum((ET.ei_s1_by_year+if(is_nan(ET.ei_s2_by_year),0.0,ET.ei_s2_by_year)) * P.production_by_year) over (order by P.year) as cumulative_trajectory,
       concat(ET.ei_s1_by_year_units, ' * ', P.production_by_year_units) as cumulative_trajectory_units,
       sum((EI.ei_s1_by_year+if(is_nan(EI.ei_s2_by_year),0.0,EI.ei_s2_by_year)) * P.production_by_year) over (order by P.year) as cumulative_target,
       concat(EI.ei_s1_by_year_units, ' * ', P.production_by_year_units) as cumulative_target_units
from demo_dv.template_company_data C
     join demo_dv.template_production_data P on P.company_id=C.company_id
     join demo_dv.template_target_da

### Step 3: Visualize Emissions, Targets, and Trajectories

SuperSet Dashboard here (not really, not yet, but points to TRINO_USER dashboard, not TRINO_USER1 dashboard): https://superset-secure-odh-superset.apps.odh-cl2.apps.os-climate.org/superset/dashboard/4/?edit=true&native_filters=%28%29

Plot emissions data.  Others can be plotted by following same pattern.

Note that without units, a company that emits 80 t CO2e/t Steel looks like it might emit a lot more than one that emits 10t CO2e/MWh.  With units, it becomes clear that the 80 and the 10 are not comparable.

In [ ]:
sql_df = pd.read_sql_table(f"{itr_prefix}emissions_data", engine_dev)
sql_df

In [ ]:
df = requantify_df(sql_df.dropna(), typemap={'co2_s1_by_year':'Mt CO2', 'co2_s2_by_year':'Mt CO2', 'co2_s3_by_year':'Mt CO2'}).convert_dtypes()
df = df[df.company_id.ne('US6362744095+Gas Utilities') & df.company_id.ne('US0236081024+Gas Utilities') ]
df = df[df.co2_s1_by_year.gt(Q_(10.0, 'Mt CO2e'))]

In [ ]:
df.dtypes

In [ ]:
df = df.sort_values(['company_name', 'year']).reset_index(drop=True)

In [ ]:
df

In [ ]:
ureg.setup_matplotlib(True)
plottable_df = df.pivot(index='year', columns='company_name', values='co2_s1_by_year').reset_index()
# Must plot the first few columns, but then plot 1/3rd of the companies so as not to over-clutter the graph
plottable_df.plot(x='year', kind='line', figsize=(24,10))

## From this point forward, this is just a sketch...

While we could technically instantiate Quant and User engines and demonstrate the access restrictions particular to those personna, it's not really "right" to demonstrate restrictions via the database that can still be accessed via the python namespace.  So we actually put the following code into separate notebooks which cannot access the data in dataframes above.

### Step 4: Use Quant engine to access and visualize temperature scores

When the Data Vault is ready to be implemented, we can demonstrate that the Quant engine does not have access to primary company data (neither financial nor production)

Show that we *cannot* access fundamental company data (cannot show until op1st team changes permissions)

Show that we *can* access both cumulative emissions (input) and temperature scores (output)

### Step 5: Show per-company temperature score and weighted portfolio alignment score

Portfolio weighting scores (which ultimately influence portfolio alignment score) include:
* WATS (size of portfolio company positions used as weights)
* TETS (size of total emissions of portfolio companies used as weights)
* Financial fundamental weights:
    * Market Cap
    * Enterprise Value
    * Assets
    * Revenues

We can pass a list of company IDs to the Data Vault to get back a sum without exposing granular data

Show that we *cannot* access fundamental company data (cannot show until op1st team changes permissions)

Show that we *can* access both cumulative emissions (input) and temperature scores (output)

### Calculate portfolio alignment temperature score based on WATS

We can do this with information exclusive to the user space (and the probability-adjusted temperature scores)

Note that companies with no production information (such as TITAL INTERNATIONAL INC and UNIVERSAL STAINLESS & ALLOY PRODUCTS INC will show NaN (Not a Number) as a score.

### Calculate portfolio alignment temperature score based on TETS

We need to carefully meld portfolio data with corp fundamental data (in this case, emissions)

### Calculate portfolio alignment temperature score based on MOTS, EOTS, ECOTS, AOTS, and ROTS

* MOTS = market cap weights
* EOTS = enterprise value weights
* ECOTS = EVIC weights
* AOTS = asset weights
* ROTS = revenue weights

### Companies for which we lack production data (and thus cannot chart)